**Name:** Stav Yosef

**ID:** 316298876

**Colab:** [https://drive.google.com/file/d/1QpBrvWYaW58PTS5bgKhTRJ_Y-HqkfMVu/view?usp=sharing](https://drive.google.com/file/d/1QpBrvWYaW58PTS5bgKhTRJ_Y-HqkfMVu/view?usp=sharing)

##  <span style="color:blue">Exercise 3 - Driver file </span>
## <span style="color:blue">Computer Vision - Fall 2020


**Lecturer:** Prof. Yael Moses, IDC

**TA:** Eyal Friedman, IDC

**Sybmission date: 11.1.2021**



In this exercise you will practice working with videos, and simple segmentations.

## Submission guidelines:

1. Your zip file should include the following files only:
    - ex2-driver.ipynb  **Or**  ex2-driver.py 
    - ex2_ID_ID.doc  **Or**  ex2_ID_ID.pdf
2. The results you are asked to display and the open questions should be answered in a doc/pdf file. 
   (Don't add the python code to that file.)
4. You may use any IDE  (e.g., Spyder, Jupyter Notebook, Pycharm, ect.).
5. Name the file 'ex2_ID_ID.zip' and do **not** include any additional directories. 
6. Submit using *moodle*
7. Submit on time!
8. You can submit this assignment in pairs (no triples).

## Read the following instructions carefully:
1. You are responsible for the correctness of your code and should add as many tests as you see fit. Do not submit your tests, unless requested.
3. Use `python 3` and `numpy 1.18.5`. Changes of the configuration we provided are at your own risk. Before submitting the exercise, restart the kernel and run the notebook from start to finish to make sure everything works.
4. You are allowed to use functions and methods from the [Python Standard Library](https://docs.python.org/3/library/) and [numpy](https://www.numpy.org/devdocs/reference/) only. Any other imports are forbidden, unless been provided by us.
4. Your code must run without errors. Note,  **code that fails to  run will not be graded.**
5. Document your code properly.
6. **Note:** you are given a set of videos, you are welcome to use them or any other videos. If they are too long, you can use only part of the frames. If they are too large, you can rescale them.
7. In case there  are several possible variations for implementing the algorithms - make your own  choice, and give a short explanation.

## Honor Code:
The assignment is a basic tool for learning the material. You can probably find the solution on the Web. However, if  you do so, then you will not learn what you should learn from it. In addition, since we  grade  the assignment, using existing solutions will be considered dishonest.
In particular, you are not allowed to copy or use any code that solves the task. 
You are more than welcome to talk with your friends, but you are not allowed to give your code or answers and you are not allowed to use their code or answers. 
Remember – you take this course in order to learn.


In [1]:
import cv2

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
from scipy.linalg import null_space
from scipy.signal import convolve2d
from scipy.ndimage import gaussian_filter

In [7]:
import platform
print("Python version: ", platform.python_version())
print("Numpy version: ", np.__version__)

Python version:  3.6.9
Numpy version:  1.19.4


In [8]:
def get_dict_video_files() -> dict:
    return {
        'OneLeaveShopReenter2cor':
            {
                'input': "OneLeaveShopReenter2cor.mpg",
                'output_a_1': "OneLeaveShopReenter2cor_a_1.avi",
                'output_a_2': "OneLeaveShopReenter2cor_a_2.avi"
            },
        'SLIDE':
            {
                'input': "SLIDE.avi",
                'output_a_1': "SLIDE_a_1.avi",
                'output_a_2': "SLIDE_a_2.avi",
            },
        'marple7':
            {
                'input': "marple7.avi",
                'output_a_1': "marple7_a_1.avi",
                'output_a_2': "marple7_a_2.avi",
            },
        'horses':
            {
                'input': "horses.avi",
                'output_a_1': "horses_a_1.avi",
                'output_a_2': "horses_a_2.avi",
            }
        ,
        'cars5':
            {
                'input': "cars5.avi",
                'output_a_1': "cars5_a_1.avi",
                'output_a_2': "cars5_a_2.avi",
            }

    }

In [9]:
def save_output(v_foreground: np.ndarray, out_filename_video: str):
    y = v_foreground.shape[2]
    x = v_foreground.shape[1]

    out = cv2.VideoWriter(out_filename_video, cv2.VideoWriter_fourcc(*"MJPG"), 30, (y, x))

    for i in range(len(v_foreground)):
        out.write(v_foreground[i].astype('uint8'))

    out.release()

## Section A: Change Detection

**A1. Simple change detection**

Compute a simple change detection algorithm. Use as background the median of a set of k1 frames, and update it every k2 frames. Your algorithm should work on color images. Think how to merge the different channels (colors). You can assume that the camera is static. The output is a video where the pixels of the  foreground objects consists of the original frame, and the other pixels are black. 

*Input:* name_file, k1, k2, and any other parameter you would like to add\
name_file: a name of a video file\
k1: the number of frames for computing the median\
k2: the number of frames between two updates of the background

*output:*  v_foreground

In [4]:
def median_change_dection(name_file: str, k1: int = 10, k2: int = 10) -> np.ndarray:
    """
    :param name_file: Video filename (path)
    :param k1: The number of frames for computing the median.
    :param k2: The number of frames between two updates of the background.
    :return: Reconstructed video with background and foreground objects.
    """

    frames = None  # K1 frames

    idx_change = 0  # Counting from 0 to K1, when reaching to k1 restarting to 0 (updating the frames list).
    update_counter = 0  # Couting from 0 to K2, when reaching to k2 initialize background update and restarting to 0.
    frames_counter = 0  # Total frames counted in total.

    threshold = 0.25  # Threshold for the background range
    lower_bound_bg = None  # Lower boundray for all pixels
    higher_bound_bg = None  # Lower boundray for all pixels

    cap = cv2.VideoCapture(name_file)

    rows, cols = -1, -1

    v_foreground = []

    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            if frames_counter == 0:
                # Initializing video resolution.

                rows = frame.shape[0]
                cols = frame.shape[1]

                # K1 frames, each by the size (rows, cols). each pixel is RGB. (CV2 -> BGR)
                frames = np.zeros((k1, rows, cols, 3))

            frames_counter += 1

            frames[idx_change] = frame
            idx_change += 1

            if k1 == idx_change:
                # If true restart the counting and start to override the first frame.

                idx_change = 0

            update_counter += 1
            if frames_counter < k2:
                # Assume k2 = 20,
                # we don't want to lose all the first frames so we calculating each iteration new background.

                res = np.median(frames[0: frames_counter, :, :, :], axis=0)

                lower_bound_bg = (1 - threshold) * res
                higher_bound_bg = (1 + threshold) * res

            if update_counter == k2:
                # If true restart the counting and update the background.

                res = np.median(frames, axis=0)
                lower_bound_bg = (1 - threshold) * res
                higher_bound_bg = (1 + threshold) * res
                update_counter = 0

            """
            Foreach pixel we checking if each channel is in the background's channel range.
            If one of the channels is in the range so we mark it as background.
            
            It's important to note that we can choose constrain at least 2 but then we'll see a lot of noise.
            Checked both approaches and chose this one, It is all about trade off in the end.
            """

            and_1 = np.logical_and(frame[:, :, 0] > lower_bound_bg[:, :, 0], frame[:, :, 0] < higher_bound_bg[:, :, 0])
            and_2 = np.logical_and(frame[:, :, 1] > lower_bound_bg[:, :, 1], frame[:, :, 1] < higher_bound_bg[:, :, 1])
            and_3 = np.logical_and(frame[:, :, 2] > lower_bound_bg[:, :, 2], frame[:, :, 2] < higher_bound_bg[:, :, 2])
            r = np.logical_or(np.logical_or(and_1, and_2), and_3)

            # Setting pixels as background
            frame[r, :] = 0

            v_foreground.append(frame.copy())

            # cv2.imshow('frame', frame)
            # cv2.waitKey(1)
        else:
            break

    cap.release()

    return np.array(v_foreground)

In [ ]:
def a_1():
    videos_files = get_dict_video_files()
    # for key in videos_files.keys():
    key = "SLIDE"
    # key = "OneLeaveShopReenter2cor"

    v_foreground = median_change_dection(name_file=videos_files[key]['input'], k1=10, k2=10)
    save_output(v_foreground=v_foreground,
                out_filename_video=videos_files[key]['output_a_1'])

In [ ]:
a_1()

**A2. Post Processing for  change detection**

**Answer:** Suggest a post processing algorithm for improving  the results of a change detection algorithm (e.g., remove noise or fill gaps). 

**Code:** implement your algorithm.

*Input:*  v_original, v_foreground\
v_original: the original video\
v_foreground: the output of B1

*output:* v_PP_foreground\
v_PP_foreground: the result of the post processing on v_foreground.


**Note:**
1. You may want to generate from v_foreground  a binary mask of the foreground regions.
2. You can use dilation or erosion on a the binary mask.
3.  You may use additional frames to improve the results, but you do not have to.


In [ ]:
def improve_foreground(v_original: str, v_foreground: str) -> np.ndarray:
    """
    :param v_original: The original video.
    :param v_foreground: The output of B1.
    :return: The result of the post processing on v_foreground.
    """

    cap_org = cv2.VideoCapture(v_original)
    cap_v_fg = cv2.VideoCapture(v_foreground)

    rows, cols = -1, -1

    v_foreground_improved = []

    frames_counter = 0
    while cap_org.isOpened() and cap_v_fg.isOpened():
        ret_org, frame_org = cap_org.read()
        ret_v_fg, frame_v_fg = cap_v_fg.read()

        if ret_org and ret_v_fg:
            frames_counter += 1

            """
            I'm assuming that most of the pictures and videos (after our algo A.1)
            where they have areas of pixels with the color (<20, <20, <20) are probababliy noise. 
            """

            con1 = np.logical_and(frame_v_fg[:, :, 0] > 20, frame_v_fg[:, :, 1] > 20)
            con2 = np.logical_and(con1, frame_v_fg[:, :, 2] > 20)

            arr_thre = np.array(con2, dtype=np.uint8)

            # Getting Connected Components With Stats. Stats here is the key.
            output = cv2.connectedComponentsWithStats(arr_thre, connectivity=8)

            labels = output[0]
            comps = output[1]
            stats = output[2]

            labels_arrange = np.arange(labels)
            labels_arrange = labels_arrange.reshape((len(labels_arrange), 1))

            # Adding new column to stats matrix.
            stats = np.append(stats, labels_arrange, axis=1)

            # Column 5 (stats[:, 4]) is the size of each component, if its smaller than 4 then delete it.
            stats_sorted = stats[np.flip(np.argsort(stats[:, 4]))]
            stats_sorted = stats_sorted[np.where(stats_sorted[:, 4] > 3)]
            # Removing the biggest component which is the background (all zeros).
            stats_sorted = stats_sorted[1:]

            # Restoring with the new column we added above all the areas (comps) that are bigger than 3. (>= 4)
            img_remove_noise = np.zeros(con2.shape, np.uint8)
            for label in stats_sorted[:, -1]:
                img_remove_noise[np.where(comps == label)] = 1

            # After removing a lot of noise let's Dialte!
            kernel = np.ones((5, 5), np.uint8)
            no_zero_arr: np.ndarray = cv2.dilate(img_remove_noise, kernel, iterations=2)

            # Restoing all the pixels from the original frame.
            y_idx, x_idx = no_zero_arr.nonzero()

            improved_frame = np.zeros(frame_v_fg.shape, dtype=np.uint8)
            improved_frame[y_idx, x_idx] = frame_org[y_idx, x_idx]

            # cv2.imshow('frame', frame_v_fg)
            # cv2.waitKey(1)
            # print(frames_counter)
            v_foreground_improved.append(improved_frame)
        else:
            break

    cap_org.release()
    cap_v_fg.release()

    return np.array(v_foreground_improved)

In [ ]:
def a_2():
    videos_files = get_dict_video_files()
    # for key in videos_files.keys():
    key = "SLIDE"
    # key = "OneLeaveShopReenter2cor"

    v_PP_foreground = improve_foreground(v_original=videos_files[key]['input'],
                                         v_foreground=videos_files[key]['output_a_1'])
    save_output(v_foreground=v_PP_foreground,
                out_filename_video=videos_files[key]['output_a_2'])

In [ ]:
a_2()

**A3. Counting  foreground objects**

Write a function that counts the number of foreground objects in the result of A1 or A2. 

*Input:* v_foreground\
v_foreground: a video which is the output of B1 or B2

*Output*: c\
c: a vector with the number of foreground objects in each of the frames

**Note:** You can use a function that counts connected components in a binary image.


In [ ]:
def count_foreground_objects(v_foreground: str, thre_mean: int) -> np.ndarray:
    """
    :param v_foreground: A video which is the output of A1 or A2
    :param thre_mean: Threshold for the estimator, see explanation in the Report.docx
    :return: A vector with the number of foreground objects in each of the frames
    """

    cap = cv2.VideoCapture(v_foreground)

    comps_sizes_frames = []  # Storing for each frame all components size.

    frames_counter = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            frames_counter += 1

            """
            Same logic goes here as A.2.
            """

            con1 = np.logical_and(frame[:, :, 0] > 20, frame[:, :, 1] > 20)
            con2 = np.logical_and(con1, frame[:, :, 2] > 20)

            arr_thre = np.array(con2, dtype=np.uint8)

            output = cv2.connectedComponentsWithStats(arr_thre, connectivity=8)

            stats = output[2]

            stats_sorted = stats[np.flip(np.argsort(stats[:, 4]))]
            stats_sorted = stats_sorted[np.where(stats_sorted[:, 4] > 3)]
            stats_sorted = stats_sorted[1:]

            if stats_sorted.shape[0] == 0:
                comps_sizes_frames.append(np.array([]))
                continue

            comps_sizes_frames.append(stats_sorted[:, 4])

            # print(frames_counter)
        else:
            break

    cap.release()

    """
    Next I'll count the objects in a way that similar to a lot of estimators in the field, Average of means.
    The algorithm down below goes like this:
        1.	Take the components size average of each frame.
        2.	After 30 (k) frames calculate the average of the 30 frames from 1.
        3.	For each frame check how many of the components size are bigger than the  estimator (2.).
    """

    c = []
    avgs = []
    comps_sizes = []
    counter = 0
    k = 30
    # thre_mean = 1.5
    for comp_sizes in comps_sizes_frames:
        if len(comp_sizes) == 0:
            avgs.append(0)
        else:
            avgs.append(np.mean(comp_sizes))

        comps_sizes.append(comp_sizes)

        counter += 1
        if counter == k:
            mean = np.mean(avgs) * thre_mean

            for comp in comps_sizes:
                c.append((comp > mean).sum())

            counter = 0
            avgs = []
            comps_sizes = []

    # If there are 95 frames so avgs should be 5, 95 % (k = 30) = 5
    if len(avgs) > 0:
        mean = np.mean(avgs) * thre_mean

        for comp in comps_sizes:
            c.append((comp > mean).sum())

    return np.array(c)

In [ ]:
def a_3():
    videos_files = get_dict_video_files()
    for key in videos_files.keys():
        # key = "SLIDE"
        key = 'OneLeaveShopReenter2cor'

        thre_mean = 0.5
        c = count_foreground_objects(v_foreground=videos_files[key]['output_a_2'], thre_mean=thre_mean)
        print("-" * 10, key, 'A.2', "-" * 10)
        print('Threshold:', thre_mean, '(multiplying our estimator)')
        print("Objects in the video:", np.round(np.mean(c), 1))
        print("Frames:\n", c)
        break

In [ ]:
a_3()

## Section B: Compute Optical Flow (OF) using Lucas-Kanade

In [10]:
def gaussian_dx(x, y, sigma):
    return (-x / (2 * np.pi * sigma ** 4)) * np.exp(-(np.square(x) + np.square(y)) / (2 * sigma ** 2))


def gaussian_dy(x, y, sigma):
    return (-y / (2 * np.pi * sigma ** 4)) * np.exp(-(np.square(x) + np.square(y)) / (2 * sigma ** 2))


def calc_mask_size(sigma) -> int:
    def gaussian(_x, _y, _sigma):
        return (1 / (2 * np.pi * _sigma ** 2)) * np.exp(-(np.square(_x) + np.square(_y)) / (2 * _sigma ** 2))

    size_mask = 1
    while True:
        if (size_mask % 2) == 0:
            size_mask = size_mask + 1

        ax = np.linspace(-(size_mask - 1) / 2., (size_mask - 1) / 2., size_mask)

        x, y = np.meshgrid(ax, ax)

        if np.sum(gaussian(x, y, sigma)) >= 0.95:
            return size_mask
        size_mask += 1


def gaussian_derivative_xy(sigma) -> (np.ndarray, np.ndarray):
    mask_size = calc_mask_size(sigma)

    ax = np.linspace(-(mask_size - 1) / 2., (mask_size - 1) / 2., mask_size)

    x, y = np.meshgrid(ax, ax)
    g_dx = gaussian_dx(x, y, sigma)
    g_dy = gaussian_dy(x, y, sigma)

    return g_dx, g_dy



**B1. Basic Lucas Kanade OF**

Impelment the basic Lucas-Kanade we leared in class.

*Input:*  name_file, nf1, nf2,  sigma_R, sigma_S\
name_file: a name of a video file\
f1 and f2: the numbers of the two frames form the video on which the OF is computed.\
sigma_S: the variance of the Gaussian used for the  spatial smoothing  as in HW1 (for computing the derivative of a Gaussian).\
sigma_R: the variance of the Gaussian for computing the sum of the derivatives (the convolution replace the sum).

*Output:* U, V, im1,im2\
U, V:  two matrices with the x and y motion for each pixel, respectively.\
im1, im2: the frames on which the optical flow was computed (their number in the video is nf1 and nf2).

**Note:**
1. You can use any video reading method you find convenient.
2. Do not forget to convert the images into grey scale.
3. You can compute the derivatives of the images as in HW1 - convolution with the derivative of a Gaussian.
3. You can resize the images in order for the program to run faster.
5. The computed OF is not necessarily integers. You may want to perform float computation.
6. For sigma_R look at slide 63 of Class 7.

In [15]:
def basic_LK_OF(name_file: str, nf1: int, nf2: int, sigma_S: float, sigma_R: float, video_name: str) -> (
        np.ndarray, np.ndarray, np.ndarray, np.ndarray):
    """
    :param name_file: A name of a video file.
    :param nf1: Frame number in the video.
    :param nf2: Frame number in the video.
    :param sigma_S: The variance of the Gaussian used for the spatial smoothing.
    :param sigma_R: The variance of the Gaussian for computing the sum of the derivatives
    :param video_name: Video name (SLIDE, OneLeaveShopReenter2cor ect)
    :return:
    U, V: two matrices with the x and y motion for each pixel, respectively.
    im1, im2: the frames on which the optical flow was computed (their number in the video is nf1 and nf2).
    """

    cap = cv2.VideoCapture(name_file)

    tmp_1 = min(nf1, nf2)
    tmp_2 = max(nf1, nf2)

    nf1 = tmp_1
    nf2 = tmp_2

    frame_1 = None
    frame_1_rgb = None
    frame_2 = None
    frame_2_rgb = None

    rows, cols = None, None

    # Getting the frames from the video.
    frames_counter = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            if frames_counter == nf1:
                frame_1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                frame_1_rgb = frame

            if frames_counter == nf2:
                frame_2 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                frame_2_rgb = frame

            if frame_1 is not None and frame_2 is not None:
                break

            frames_counter += 1

        else:
            break

    cap.release()

    if frame_1 is None or frame_2 is None:
        return None, None, None, None

    """
    For each video I'm using different window size, not because of the content of the video, because its dimensions.
    camera: 30
    slide: 50
    """
    w_size = 30
    w = w_size // 2

     # Calculating gaussians & using them.
    g_dx_s, g_dy_s = gaussian_derivative_xy(sigma_S)
    g_dx_r, g_dy_r = gaussian_derivative_xy(sigma_R)

    mode = 'same'

    f1_i_x = convolve2d(frame_1, g_dx_s, mode=mode)
    f1_i_y = convolve2d(frame_1, g_dy_s, mode=mode)

    f2_i_x = convolve2d(frame_2, g_dx_s, mode=mode)
    f2_i_y = convolve2d(frame_2, g_dy_s, mode=mode)

    ft = gaussian_filter(frame_1, sigma_S) - gaussian_filter(frame_2, sigma_S)

    # Preparation for C matrix.
    f1_i_x_pow = convolve2d(np.power(f1_i_x, 2), g_dx_r, mode=mode)
    f1_i_y_pow = convolve2d(np.power(f1_i_y, 2), g_dy_r, mode=mode)
    f1_i_x_i_y = convolve2d(f1_i_x * f1_i_y, g_dx_r, mode=mode)

    # Initializing
    u, v = np.zeros(frame_1.shape), np.zeros(frame_1.shape)
    _rows, _cols = frame_1.shape

    for i in range(w, _rows - w):
        print(f'{i}/{_rows - w}')
        for j in range(w, _cols - w):
            # Building C matrix.
            c = np.array([[f1_i_x_pow[i, j], f1_i_x_i_y[i, j]],
                          [f1_i_x_i_y[i, j], f1_i_y_pow[i, j]]])

            if np.linalg.matrix_rank(c) != 2:
                continue

            # Creating A and b matrices
            i_x = f1_i_x[i - w: i + w + 1, j - w: j + w + 1].flatten()
            i_y = f1_i_y[i - w: i + w + 1, j - w: j + w + 1].flatten()
            i_t = ft[i - w: i + w + 1, j - w: j + w + 1].flatten()

            b = np.reshape(i_t, (i_t.shape[0], 1))
            a = np.vstack((i_x, i_y)).T

            a_pinv = np.linalg.pinv(a)

            uv = np.matmul(a_pinv, b)

            u[i, j] = uv[0]
            v[i, j] = uv[1]

    return u, v, frame_1_rgb, frame_2_rgb

In [ ]:
videos_files = get_dict_video_files()
key = "SLIDE"
U_SLIDE, V_SLIDE, im1_SLIDE, im2_SLIDE = basic_LK_OF(name_file=videos_files[key]['input'],
                                                     nf1=66,
                                                     nf2=68,
                                                     sigma_R=1,
                                                     sigma_S=1,
                                                     video_name=key)

In [ ]:
videos_files = get_dict_video_files()
key = "OneLeaveShopReenter2cor"
U_SHOP, V_SHOP, im1_SHOP, im2_SHOP = basic_LK_OF(name_file=videos_files[key]['input'],
                                                 nf1=80,
                                                 nf2=84,
                                                 sigma_R=1,
                                                 sigma_S=1,
                                                 video_name=key)

**B2. Present OF results**

*Input:* im1, U, V (the output of Basic_LK_OF without im2).

*Output:* a quiver plot overlaid the input  frame

**Note:**
1. You can look at https://pythonforundergradengineers.com/quiver-plot-with-matplotlib-and-jupyter-notebooks.html
2. You may want for visualaization to uniformally resize the values of U and V - if they are too large or too small/
2. You may not want to draw the OF  each pixel - to avoid OF overlapping of neighboring pixels.

In [13]:
def OF_plot_results(U: np.ndarray, V: np.ndarray, im1: np.ndarray):
    """
     U, V: two matrices with the x and y motion for each pixel, respectively.
    :param U: Matrix with the x motion for each pixel.
    :param V: Matrix with the y motion for each pixel.
    :param im1: Frame (nf1), output from B.1.
    """

    fig, ax = plt.subplots()
    fig.set_size_inches(20, 15)

    # Converting from BRG (cv2) to RGB.
    ax.imshow(cv2.cvtColor(im1, cv2.COLOR_BGR2RGB))

    _rows, _cols, _ = im1.shape

    x_pos = np.array([np.arange(_cols)])
    x_pos = np.repeat(x_pos, _rows, axis=0)

    y_pos = np.arange(_rows).reshape((_rows, 1))
    y_pos = np.repeat(y_pos, _cols, axis=1)

    """
    I want to show in pretty way all the arrows so I found the right balace.
    Can be extended to formula according to the dimensions of the video.
    camera: 200
    slide: 400
    """
    x_direct = U * 400
    y_direct = V * 400

    strength = np.sqrt(np.square(x_direct) + np.square(y_direct))

    """
    Showing every i arrow in x axis and j arrow in y axis.
    camera:   rows: 10    cols: 5
    slide:    rows: 15    cols: 30
    """
    set_to_zero_rows = np.arange(_rows).reshape((_rows, 1))
    set_to_zero_rows = np.repeat(set_to_zero_rows, _cols, axis=1)
    set_to_zero_rows = np.where(set_to_zero_rows % 15 != 0)
    strength[set_to_zero_rows] = 0

    set_to_zero_cols = np.arange(_cols).reshape((1, _cols))
    set_to_zero_cols = np.repeat(set_to_zero_cols, _rows, axis=0)
    set_to_zero_cols = np.where(set_to_zero_cols % 30 != 0)
    strength[set_to_zero_cols] = 0

    st = strength[strength.nonzero()]

    # Taking indices of all the arrows with a change.
    threshold = np.mean(st)
    indices = np.where(np.abs(strength) > 0)

    # Taking only arrows that bigger than the average.
    valid_arrows_count = len(np.where(np.abs(strength) > threshold)[0])

    strength = strength[indices]

    x_direct = x_direct[indices]
    y_direct = y_direct[indices]

    x_pos = x_pos[indices]
    y_pos = y_pos[indices]

    """
    Next is something custom I made for better understanding which arrow is strong and which is not.
    The strong arrows will have high alpha (color transparency) and weak low alpha.   
    """
    c = np.array([colors.to_rgba([0, 0, 0], alpha=1)])

    _colors = np.repeat(c, len(x_pos), axis=0)

    _sorted = np.flip(np.argsort(strength))

    step = 0.8 / valid_arrows_count
    alpha = 1.0
    counter = 0
    # step = 0
    for arrow in _sorted:
        if counter > valid_arrows_count:
            _colors[arrow][3] = 0
        else:
            _colors[arrow][3] = alpha
            alpha -= step

        counter += 1

    # units="width/xy"
    # ax.quiver(x_pos, y_pos, x_direct, y_direct, scale=10, color=_colors, units="xy")
    ax.quiver(x_pos, y_pos, x_direct, y_direct, scale=10, color=_colors, units="xy")

    ax.axis([0, _cols, 0, _rows])
    ax.set_aspect('equal')

    plt.gca().invert_yaxis()
    plt.show()

In [ ]:
OF_plot_results(U=U_SLIDE, V=-V_SLIDE, im1=im1_SLIDE)

In [ ]:
OF_plot_results(U=U_SHOP, V=-V_SHOP, im1=im1_SHOP)

**B3. Evaluate OF results**

*Input:* im1, im2,  U, V (the output of Basic_LK_OF).

*Output:* w_im1, w_diff, err\
w_im1: the results of wrapping im1 using (U,V) toward im2 (matrix).\
w_diff:  |wraped_im1 -  im2| (matrix).\
err: the sum of square differences between w_im1 and im2 (scalar).

**Note:**\
 You can use any wrapping function you like from openCV or other code.


In [21]:
def evaluate_OF_results(im1: np.ndarray, im2: np.ndarray, U: np.ndarray, V: np.ndarray) -> (
        np.ndarray, np.ndarray, float):
    im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2RGB)
    im2 = cv2.cvtColor(im2, cv2.COLOR_BGR2RGB)

    x_direct_pixels = np.array(np.round(U * 20, decimals=0), dtype=np.int)
    y_direct_pixels = np.array(np.round(V * 20, decimals=0), dtype=np.int)

    im1_to_im2 = im1.copy()

    # print(np.mean(U))
    # print(np.mean(V))

    U_sorted = np.sort(np.sqrt(np.square(U) + np.square(V)), axis=1)

    strength = np.sqrt(np.square(U) + np.square(V))

    y_idx, x_idx = np.where(strength > 0.1)

    for i in range(y_idx.shape[0]):
        y = y_idx[i]
        x = x_idx[i]

        change_x = x_direct_pixels[y, x]
        change_y = y_direct_pixels[y, x]

        im1_to_im2[y + change_y, x + change_x] = im1[y, x]

    w_diff = np.abs(im1_to_im2 - im2)
    err = np.sum(np.power(np.array(im1_to_im2, dtype=np.float32) - np.array(im2, dtype=np.float32), 2))
    # plt.imshow(w_diff)
    # plt.imshow(im1_to_im2)
    # plt.show()

    return im1_to_im2, w_diff, err

In [27]:
im1_to_im2_SLIDE, w_diff_SLIDE, error_SLIDE = evaluate_OF_results(im1=im1_SLIDE, im2=im2_SLIDE, U=U_SLIDE, V=V_SLIDE)

In [30]:
error_SLIDE

892727360.0

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20, 15)

ax.imshow(im1_to_im2_SLIDE)

plt.show()

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20, 15)

ax.imshow(w_diff_SLIDE)

plt.show()

In [31]:
im1_to_im2_SHOP, w_diff_SHOP, error_SHOP = evaluate_OF_results(im1=im1_SHOP, im2=im2_SHOP, U=U_SHOP, V=V_SHOP)

In [32]:
error_SHOP

91407030.0

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20, 15)

ax.imshow(im1_to_im2_SHOP)

plt.show()

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20, 15)

ax.imshow(w_diff_SHOP)

plt.show()

**B4. Affine_LK_OF**

Use the variant of Lucas-Kanade with affine motion instead of translation.
See slides - class 7 slides 73-75.

The input and output is the same as in **B1**.



In [36]:
def affine_LK_OF(name_file: str, nf1: int, nf2: int, sigma_S: float, sigma_R: float, video_name: str) -> (
        np.ndarray, np.ndarray, np.ndarray, np.ndarray):
    """
    :param name_file: A name of a video file.
    :param nf1: Frame number in the video.
    :param nf2: Frame number in the video.
    :param sigma_S: The variance of the Gaussian used for the spatial smoothing.
    :param sigma_R: The variance of the Gaussian for computing the sum of the derivatives
    :param video_name: Video name (SLIDE, OneLeaveShopReenter2cor ect)
    :return:
    U, V: two matrices with the x and y motion for each pixel, respectively.
    im1, im2: the frames on which the optical flow was computed (their number in the video is nf1 and nf2).
    """

    cap = cv2.VideoCapture(name_file)

    tmp_1 = min(nf1, nf2)
    tmp_2 = max(nf1, nf2)

    nf1 = tmp_1
    nf2 = tmp_2

    frame_1 = None
    frame_1_rgb = None
    frame_2 = None
    frame_2_rgb = None

    rows, cols = None, None

    # Getting the frames from the video.
    frames_counter = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            if frames_counter == nf1:
                frame_1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                frame_1_rgb = frame

            if frames_counter == nf2:
                frame_2 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                frame_2_rgb = frame

            if frame_1 is not None and frame_2 is not None:
                break

            frames_counter += 1

        else:
            break

    cap.release()

    if frame_1 is None or frame_2 is None:
        return None, None

    """
    For each video I'm using different window size, not because of the content of the video, because its dimensions.
    camera: 30
    slide: 50
    """

    w_size = 50
    w = w_size // 2

    # Calculating gaussians & using them.
    g_dx_s, g_dy_s = gaussian_derivative_xy(sigma_S)
    g_dx_r, g_dy_r = gaussian_derivative_xy(sigma_R)

    mode = 'same'

    f1_i_x = convolve2d(frame_1, g_dx_s, mode=mode)
    f1_i_y = convolve2d(frame_1, g_dy_s, mode=mode)

    f2_i_x = convolve2d(frame_2, g_dx_s, mode=mode)
    f2_i_y = convolve2d(frame_2, g_dy_s, mode=mode)

    ft = gaussian_filter(frame_1, sigma_S) - gaussian_filter(frame_2, sigma_S)

    # Preparation for C matrix.
    f1_i_x_pow = convolve2d(np.power(f1_i_x, 2), g_dx_r, mode=mode)
    f1_i_y_pow = convolve2d(np.power(f1_i_y, 2), g_dy_r, mode=mode)
    f1_i_x_i_y = convolve2d(f1_i_x * f1_i_y, g_dx_r, mode=mode)

    # Initializing
    u = np.zeros(frame_1.shape)
    v = u.copy()

    _rows, _cols = frame_1.shape

    xu, yv = np.meshgrid(np.arange(0, _cols), np.arange(0, _rows))

    for i in range(w, _rows - w):
        print(f'{i}/{_rows - w}')
        for j in range(w, _cols - w):
            i_x = f1_i_x[i - w: i + w + 1, j - w: j + w + 1].flatten()
            i_y = f1_i_y[i - w: i + w + 1, j - w: j + w + 1].flatten()
            i_t = ft[i - w: i + w + 1, j - w: j + w + 1].flatten()
            _x = xu[i - w: i + w + 1, j - w: j + w + 1].flatten()
            _y = yv[i - w: i + w + 1, j - w: j + w + 1].flatten()

            b = np.reshape(i_t, (i_t.shape[0], 1))
            a = np.vstack((i_x, _x * i_x, _y * i_x, i_y, _x * i_y, _y * i_y)).T
            c = np.matmul(np.transpose(a), a)

            if np.linalg.matrix_rank(c) != 6:
                continue

            a_pinv = np.linalg.pinv(a)

            uv = np.matmul(a_pinv, b)

            u[i, j] = uv[0] + uv[1] * j + uv[2] * i
            v[i, j] = uv[3] + uv[4] * j + uv[5] * i

    return u, v, frame_1_rgb, frame_2_rgb

In [ ]:
videos_files = get_dict_video_files()
key = "SLIDE"
affine_U_SLIDE, affine_V_SLIDE, affine_im1_SLIDE, affine_im2_SLIDE = affine_LK_OF(name_file=videos_files[key]['input'],
                                                                                  nf1=66,
                                                                                  nf2=68,
                                                                                  sigma_R=1,
                                                                                  sigma_S=1,
                                                                                  video_name=key)

In [ ]:
videos_files = get_dict_video_files()
key = "OneLeaveShopReenter2cor"
affine_U_SHOP, affine_V_SHOP, affine_im1_SHOP, affine_im2_SHOP = affine_LK_OF(name_file=videos_files[key]['input'],
                                                                              nf1=80,
                                                                              nf2=84,
                                                                              sigma_R=1,
                                                                              sigma_S=1,
                                                                              video_name=key)

In [ ]:
OF_plot_results(U=affine_U_SLIDE, V=-affine_V_SLIDE, im1=affine_im1_SLIDE)

**B5. Apply and Discuss:**

Run the basic_LK_OF, and the affine_LK_OF on one or two videos, one with a static camera and the other with a moving camera.
Play with the frames chosen from each video, the algorithm parameters, and  the distance between nf1 and nf2.

**Answer:**
1. The disparity you compute in HW2 were integers while the OF is not necessarily integer. Expalin why. 
2. Explain theoretically for which regions the basic_LK_OF is expected to give good results and for which it does not.
2. Demonstrate your answer to (2) by displaying the results of OF overlaid im1  (Quiver overlayed im1), and mark good and bad results.
3. Explain theoretically when the basic_LK_OF is expected to fail while affine_LK_OF works well.
4. Find an example for (4)  (at least a region in the scene) and display it.
5. When two OF vectors have the same magnitude, are they necessarily corresponds to 3D points that moves at the same speed?


## Section C: Segementation  -- Not for submission

Part C will not be graded, hence,  **you do not have to submit it**. In case you would like to get feedback on it anyway - you are welcome to submit it.

**C1. Simple OF segemntation**

Use a simple segmentation (e.g., threshold) for the OF results based on the OF magnitude.

*Input:* U, V, any other parameters you find necessary
U, V - the OF vectors (e.g., computed by  B1 or B4).

*Output:* segments
im_segments is an image in which each segment is colored by a different color.


In [ ]:
def im_segments = simple_segment_OF(U,V, "any other paramers")
   
    # Your code

**C2. K-means OF segemntation**

Use k-mean segmentation for the OF results based on the OF magnitude and directions.

*Input:* U, V, any other parameters you find necessary\
U, V - the OF vectors (e.g., computed by  B1 or B4).

*Output:* im_segments\
im_segments is an image in which each segment is colored by a different color.

In [ ]:
def im_segments = simple_segment_OF(U,V, "any other paramers")
   
    # Your code

**C3. Apply  and answer**

1. Apply the functions in C1 and C2 on a video of your choice with a moving camera.
2. Discuss which method (C1 or C2) works better. Give an example.


**C4. OF used for change detection**

Assume that the video is taken by a static camera and use the results of C1 or C2 on the output in order to detect moving regions in the scene.

*input:* name_file, nf1, "any parameters you need"\
name_file: a name of a video file.\
nf1: the frame on which the OF is computed (nf1 and nf1+1).

*output*: v_change\
v_foreground: as in A1



In [ ]:
def [v_foreground] = OF_change_detection(name_file, nf1, "any parameters you need"):
    
    # your code

**C5. Change detection comparisons**

Choose a video taken from a static camera (one of the videos you used for change detection). 

1. Apply the functions A1 or A2 and C4.
4. Discuss which method (A1 or C4) works better. Give an example.
5. Count the number of moving regions for A1, A3, and C4. 